<a href="https://colab.research.google.com/github/Data-finest/Habibty/blob/main/Project_Dikjstra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

\
### SHORTEST PATH ALGORITHM

In [ ]:
#Importing of Libraries
import geopy.distance
import pandas as pd

#Mount file from google drive 
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
#Read csv file from google drive
pd.set_option("display.precision", 10)
# data = pd.read_csv("/content/drive/MyDrive/shortest_path/Location_CuttySark _New.csv")
data = pd.read_csv("/content/drive/MyDrive/shortest_path/Location_Cutty_Sark c (2)")
# data.head() # store the data in a csv file

data 

\
### Extracting the longitude, latitude and location into a dictionary data structure.

In [ ]:
# extractinhg the longitude, latitude and location into a dictionary data structure
subset = data[:16]
keys = subset['Location']
lats = subset['Latitude']
longs = subset['Longitude']
loc = {}
st = 65
for i in range(len(keys)):
  # keys[i] = [lats[i], longs[i]]
  loc[chr(st+i)] = [lats[i], longs[i]]

print(loc)

\
### Calculating the distance btw two points given their locations

In [ ]:
"""
  This function calculates the distance in kilometres between two points from the loc dictionary,
  given their longitudes and latitudes
  param lat1 = latitude of pos 1
  param long1 = longitude of pos 1
  param lat2 = latitude of pos 2
  param long2 = longitude of pos 2
"""
def calculate_distance(lat1, long1, lat2, long2):
  return geopy.distance.geodesic((lat1, long1), (lat2, long2)).meters
  
# calculates the distance btw two points given their locations
def calc_dist(pos1, pos2):
  return calculate_distance(loc[pos1][0], loc[pos1][1], loc[pos2][0], loc[pos2][1])

def print_dist(p1, p2):
    d = round(calc_dist(p1, p2), 4)
    print(f'Distance from {p1} to {p2} is {d} metres')
    return d
# gr.add_vertice('A', 'B', calculate_distance(loc['A'][0], loc['A'][1],loc['B'][0],loc['B'][1]))
def debug_distances(path):
    total = 0
    for i in range(len(path) - 1):
        total += print_dist(path[i], path[i+1])
    print ('Total Distance = ', total,' metres', '\n')
    return total

debug_distances(['O', 'N', 'M', 'K', 'L', 'G', 'F', 'A'])
debug_distances(['A', 'B', 'D', 'E', 'I', 'J', 'N'])

In [ ]:
dist = calc_dist('G', 'H')
print(f'Distance from G to H is {round(dist, 4)} metres')

### Data structure to store the Graph

In [ ]:
class Graph:
  def __init__(self):
    self.root = {}
    self.num_edges = 0
    self.num_vertices = 0;

  def add_edges(self, edges):
    for edge in edges:
      self.add_edge(edge)

  def add_edge(self, edge):
    if edge in self.root: 
      print('edge already in graph')
    else:
      self.root[edge] = []
      self.num_edges = self.num_edges + 1
  
  def add_vertice(self, from_edge, to_edge, weight):
    if from_edge not in self.root:
      print(f'edge {from_edge} not in graph, please add using the add_edge() method')
      return
    if from_edge not in self.root:
      print(f'destination edge {to_edge} not in graph, please add using the to_edge() method')
      return
    self.root[from_edge].append([to_edge, weight])
    self.num_vertices = self.num_vertices + 1

  def graph(self):
    return self.root

### CREATE AND POPULATE THE GRAPH

In [ ]:
# list of connections
# data format is a dictionary with the key as the source location, and values as its connections

gg = {
    'A': ['F', 'B'],
    'B': ['A', 'C', 'D'],
    'C': ['B', 'E'],
    'D': ['F', 'B', 'E'],
    'E': ['C', 'D', 'I'],
    'F': ['D', 'A', 'G'],
    'G': ['L', 'F', 'H'],
    'H': ['G', 'I', 'M'],
    'I': ['E', 'J', 'H'],
    'J': ['I', 'O'],
    'K': ['M', 'H', 'L'],
    'L': ['G', 'K'],
    'M': ['K', 'N'],
    'N': ['M', 'O'],
    'O': ['J', 'N']
}
gr = Graph()
gr.add_edges(gg.keys())
for source in gg.keys():
  # print('source: ',source)
  for dest in gg[source]:
    ds = calc_dist(source, dest)
    gr.add_vertice(source, dest, ds)
    print(f'distance from {source} to {dest} is {round(ds, 6)} metres')
print(gr.graph())

\
#Dikjstra Algorithm 

In [ ]:
### Building of the Dikjstra Algorithm
### ['A, 'B', 'E', 'F', 'G']

g = gr.graph()


def shortest_path(graph, n1, n2):
  visited = [] # store visited locations
  unvisited = list(graph.keys()) # store unvisited locations

  state = Graph()
  state.add_edges(unvisited)
  state = state.graph() # a map to act as the state for our algorithm

  def is_visited(key): # this function checks if a location has been visited
    return visited.__contains__(key)

  def find_min_distance_from_source(): # this functions computes the unvisited key with the shortest distance to A
    skey = None
    min = float('inf')
    for key in unvisited:
      if (state[key][0] < min):
        skey = key
    return skey

  for key in state.keys():
    if key == n1:
      state[key] = [0, None]
    else:
      state[key] = [float('inf'), None]
  # state = {'A': [DIST, VISITED_FROM]}
  # graph = { 'A: [[NODE, DIST_FROM_KEY_NODE]] }
  start = n1 # the node to start from

#Dikjstra Algorithm 
  while len(unvisited) > 0:
    edges = graph[start]

    for edge in edges:
      sDistFrmA = state[start][0] + edge[1] # calculate the distance from A
      if state[edge[0]][0] > sDistFrmA: # check if recorded shortest distance is greater than new shortest distance
        state[edge[0]][0] = sDistFrmA # replace if true
        state[edge[0]][1] = start # set visited from to start

    visited.append(start) # add start to visited
    unvisited.remove(start) # remove start from visited
    if start is n2:
      break;
    start = find_min_distance_from_source() # calculate min known distance from n1 and set start to value
  
  result = [n2] # add destination to result

  traceback = n2 # variable to hold back tracking from destination
  while traceback is not n1:
    prev = state[traceback][1] # get visited from for traceback
    if result.__contains__(prev): # if result contains the prev edge forcefully end the loop
      break
    if prev is None: break

    result.append(prev)
    traceback = prev
    
  

  return result

sp = shortest_path(g, 'A', 'O')

print(f'Shortest Path: {sp} ')

In [ ]:
grf = {
    'AB': [['BB', 2], ['BC', 5]],
    'BB': [['AB', 2], ['CD', 6]],
    'BC': [['CD', 4]],
    'CD': [['BC', 4], ['BB', 6]]
}

sps = shortest_path(grf, 'AB', 'CD')
print(sps)